In [62]:
from edgar_functions_V2 import *
import statistics

ticker = "NVDA"

In [63]:
def combine_or_add_columns(df, cols, add_instead=False, replace_zero=False):
    if not cols:
        return None

    primary_col = None

    for col in cols:
        if col in df.columns:
            primary_col = df[col]
            break

    if primary_col is None:
        return None

    if len(cols) == 1:
        return primary_col

    for col in cols[1:]:
        if col in df.columns:
            # Skip if the column contains all NaN values
            if df[col].isna().all():
                continue

            if add_instead:
                primary_col = primary_col.add(df[col], fill_value=0).combine_first(
                    primary_col.combine_first(df[col])
                )
            else:
                if replace_zero:
                    primary_col = primary_col.mask(primary_col == 0).combine_first(
                        df[col]
                    )
                else:
                    primary_col = primary_col.combine_first(df[col])

    return primary_col

In [64]:
annual_df = process_and_aggregate_annual_data(ticker)
income_mapping = income_statement_mapping.income_mapping
balance_mapping = balance_sheet_mapping.balance_mapping
cash_mapping = cash_flow_mapping.cash_mapping
income = get_changed_columns(income_mapping, annual_df).droplevel(1, axis=1)
balance = get_changed_columns(balance_mapping, annual_df).droplevel(1, axis=1)
cash_flow = get_changed_columns(cash_mapping, annual_df).droplevel(1, axis=1)

income_df = income.T.groupby(income.columns).mean().sort_index(ascending=False)
income_df = income_df.sort_index(axis=1, ascending=False)
income_df = income_df.T
income_df["Cost of Revenue"] = income_df["Net Revenue"] - income_df["Gross Profit"]
income_df["EBITDA"] = (
    income_df["EBT"]
    + income_df["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)

balance_df = balance.T.groupby(balance.columns).mean().sort_index(ascending=False)
balance_df = balance_df.T.sort_index(axis=1, ascending=False)
balance_df["Total Equity(calc)"] = (
    balance_df["Total Assets"] - balance_df["Total Liabilities"]
)
balance_df["Toal Liabilites and Equity(calc)"] = balance_df[
    "Total Liabilities"
] + combine_or_add_columns(balance_df, ["Total Equity", "Total Equity(calc)"])

In [65]:
balance_df

,Treasury Stock at Cost,Treasury Stock Shares,Total Stockholders Equity,Total Liabilities and Stockholders Equity,Total Liabilities,Total Current Liabilities,Total Current Assets,Total Assets,Retained Earnings,"Property, Plant, and Equipment Gross",...,Additional Paid in Capital,Accumulated Other Comprehensive Income(Loss),Accumulated OCI Derivatives,Accumulated Depreciation PPE,Accrued Income Taxes Noncurrent,Accrued Expenses,Accounts Receivable,Accounts Payable,Total Equity(calc),Toal Liabilites and Equity(calc)
2007,NaN,NaN,"2,617,912,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,"1,463,268,000","90,925,818","2,394,652,000","3,350,727,000",NaN,"778,591,000","2,167,958,000","3,350,727,000","1,964,169,000",NaN,...,"1,889,257,000","3,865,000",NaN,NaN,NaN,"559,727,000","318,435,000","218,864,000",NaN,NaN
2009,"1,463,268,000","90,925,857","2,665,140,000","3,585,918,000",NaN,"784,378,000","2,480,830,000","3,585,918,000","1,896,182,000","1,088,483,000",...,"2,219,401,000","12,172,000",NaN,"516,625,000","53,397,000","439,851,000","374,963,000","344,527,000",NaN,NaN
2010,"1,479,392,000","92,043,036","3,181,462,000","4,495,246,000",NaN,"942,682,000","3,226,950,000","4,495,246,000","2,149,328,000","1,193,939,000",...,"2,500,577,000","10,272,000",NaN,"-625,082,000","57,590,000","656,544,000","348,770,000","286,138,000",NaN,NaN
2011,"1,496,904,000","93,160,687","4,145,724,000","5,552,928,000",NaN,"929,958,000","3,905,358,000","5,552,928,000","2,730,418,000","1,156,528,000",...,"2,900,896,000","10,614,000",NaN,"596,456,000","63,007,000","594,886,000","336,143,000","335,072,000",NaN,NaN
2012,"1,622,709,000","103,397,063","4,827,000,000","6,412,245,000",NaN,"976,223,000","4,775,258,000","6,412,245,000","3,246,088,000","1,288,782,000",...,"3,193,623,000","9,981,000",NaN,"712,638,000","115,267,000","619,795,000","454,252,000","356,428,000",NaN,NaN
2013,"2,537,295,000","167,246,000","4,455,000,000","7,250,894,000",NaN,"945,496,000","5,624,711,000","7,250,894,000","3,504,742,000","1,282,820,000",...,"3,483,342,000","4,877,000",NaN,"700,080,000","119,977,000","621,105,000","426,357,000","324,391,000",NaN,NaN
2014,"3,395,000,000","214,000,000","4,418,000,000","7,201,000,000","2,783,000,000","896,000,000","5,713,000,000","7,201,000,000","3,949,000,000","1,179,000,000",...,"3,855,000,000","8,000,000",0,"622,000,000","121,000,000","603,000,000","474,000,000","293,000,000","4,418,000,000","7,201,000,000"
2015,"4,048,000,000","242,000,000","4,469,000,000","7,370,000,000","2,814,000,000","2,351,000,000","6,053,000,000","7,370,000,000","4,350,000,000","1,100,000,000",...,"4,170,000,000","-4,000,000","-4,000,000","634,000,000","78,000,000","642,000,000","505,000,000","296,000,000","4,556,000,000","7,370,000,000"
2016,"5,039,000,000","283,000,000","5,762,000,000","9,841,000,000","4,048,000,000","1,788,000,000","8,536,000,000","9,841,000,000","6,108,000,000","1,191,000,000",...,"4,708,000,000","-16,000,000",NaN,"670,000,000","96,000,000","507,000,000","826,000,000","485,000,000","5,793,000,000","9,841,000,000"


In [66]:
cash_df = cash_flow.T.groupby(cash_flow.columns).mean().sort_index(ascending=False)
cash_df = cash_df.T.sort_index(axis=1, ascending=False)
cash_df["Depreciation and Amortization(calc)"] = combine_or_add_columns(
    cash_df, ["Depreciation", "Amortization"], add_instead=True
)

In [67]:
cash_df.T.sort_index(axis=1, ascending=False)

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
Tax Withholding Related to Stock Based Compensation,"1,475,000,000","1,904,000,000","942,000,000","551,000,000","1,032,000,000","612,000,000","176,000,000","66,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tax Benefit Related to Stock Based Compensation operating,NaN,NaN,NaN,NaN,NaN,0,0,"10,000,000","18,000,000","26,000,000","68,710,000","52,793,000","15,316,000","2,034,000",NaN,NaN
Tax Benefit Related to Stock Based Compensation financing,NaN,NaN,NaN,NaN,NaN,0,0,"10,000,000","18,000,000","26,000,000","68,710,000","52,793,000","15,316,000","2,034,000","871,000",NaN
Stock Based Compensation Expense,"2,709,000,000","2,004,000,000","1,397,000,000","844,000,000","557,000,000","391,000,000","247,000,000","204,000,000","158,000,000","136,000,000","136,662,000","136,354,000","100,353,000","107,091,000","162,706,000","133,365,000"
Restructuring Charges,NaN,NaN,NaN,NaN,0,0,"8,000,000","97,000,000",0,0,NaN,NaN,NaN,NaN,NaN,NaN
Repurchases of Common Stock,"10,039,000,000",0,0,0,"1,579,000,000","909,000,000","739,000,000","587,000,000","814,000,000","887,000,000","100,000,000",0,0,0,"423,636,000","552,512,000"
Repayment of Long Term Debt,0,"1,000,000,000",0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Repayment of Convertible Notes,NaN,NaN,0,0,"16,000,000","812,000,000","673,000,000",0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Realized Foreign Exchange Loss,NaN,NaN,NaN,NaN,NaN,NaN,"5,000,000",NaN,"500,000","5,000,000","-1,500,000","1,600,000","-2,400,000","-900,000","2,000,000",NaN
"Purchase of Property, Plant, and Equipment",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"138,735,000","97,890,000","77,601,000",NaN,NaN


In [68]:
for column in cash_df.columns:
    print(column)

Tax Withholding Related to Stock Based Compensation
Tax Benefit Related to Stock Based Compensation operating
Tax Benefit Related to Stock Based Compensation financing
Stock Based Compensation Expense
Restructuring Charges
Repurchases of Common Stock
Repayment of Long Term Debt
Repayment of Convertible Notes
Realized Foreign Exchange Loss
Purchase of Property, Plant, and Equipment
Purchase of Marketable Securities
Proceeds from maturity of Marketable Securities
Proceeds from Sales and Maturities of Marketable Securities
Proceeds from Sale of Marketable Securities
Proceeds from Issuance of Warrants
Proceeds from Issuance of Long Term Debt
Proceeds from Issuance of Common Stock
Proceeds from Exercise of Stock Options
Principal Payments Under Finance Lease
Payments for Hedges
Payments for Acquisitions
Payment of Dividends
Other Non Cash Operating Activities
Other Investing Activities
Other Financing Activities
Operating Lease Payments
Non-Cash Purchases of PPE Not Yet Paid
Net Earnings
Ne

In [69]:
from numpy import negative


scorecard = pd.DataFrame()
scorecard["Net Revenue"] = income_df["Net Revenue"]
scorecard["Cost of Revenue"] = income_df["Cost of Revenue"]
scorecard["Gross Profit"] = income_df["Gross Profit"]
scorecard["Gross Margin"] = scorecard["Gross Profit"] / scorecard["Net Revenue"]
scorecard["SG&A"] = income_df["SG&A"]
scorecard["Operating Income"] = income_df["Operating Income"]
scorecard["Operating Margin %"] = (
    scorecard["Operating Income"] / scorecard["Net Revenue"]
) * 100
scorecard["Interest Expense"] = income_df["Interest Expense"]
scorecard["EBT"] = combine_or_add_columns(
    income_df, ["EBT", "EBT and equity investments"]
)
scorecard["Tax Expense"] = income_df["Taxes"]
scorecard["Tax Rate %"] = income_df["Tax Rate Continuing Operations"] * 100
scorecard["Net Income"] = income_df["Net Income"]
scorecard["EPS"] = income_df["EPS"]
scorecard["EPS Diluted"] = income_df["EPS Diluted"]
scorecard["EBITDA"] = (
    scorecard["EBT"]
    + scorecard["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)
scorecard["EBITDA Margin %"] = (scorecard["EBITDA"] / scorecard["Net Revenue"]) * 100
scorecard["Dividend"] = income_df["Declared Dividends per Share"]
scorecard["Dividend Payout Ratio %"] = (scorecard["Dividend"] / scorecard["EPS"]) * 100
scorecard["Cash"] = combine_or_add_columns(
    balance_df, ["Cash", "Cash and Restricted Cash"]
)
scorecard["Marketable Securities"] = combine_or_add_columns(
    balance_df, ["Marketable Securities"]
)
scorecard["Inventory"] = balance_df["Inventory"]
scorecard["Accounts Receivable"] = balance_df["Accounts Receivable"]
scorecard["Other Current Assets"] = balance_df["Other Current Assets"]
scorecard["Total Current Assets"] = balance_df["Total Current Assets"]
scorecard["Property, Plant, and Eqipment"] = balance_df[
    "Property, Plant, and Equipment"
]
scorecard["Right of Use Assets"] = combine_or_add_columns(
    balance_df, ["Operating Lease Right of Use Asset"]
)
scorecard["Other Long Term Assets"] = combine_or_add_columns(
    balance_df, ["Other Long Term Assets"]
)
scorecard["Goodwill"] = balance_df["Goodwill Asset"]
scorecard["Intangible Assets"] = combine_or_add_columns(
    balance_df, ["Intangible Assets"]
)
scorecard["Total Assets"] = balance_df["Total Assets"]
scorecard["Accounts Payable"] = combine_or_add_columns(balance_df, ["Accounts Payable"])
scorecard["Accrued Expenses"] = combine_or_add_columns(balance_df, ["Accrued Expenses"])
scorecard["Deferred Revenue"] = combine_or_add_columns(balance_df, ["Deferred Revenue"])
scorecard["Income Tax Payable"] = combine_or_add_columns(
    balance_df, ["Income Taxes Payable"]
)
scorecard["Operating Lease Liability"] = combine_or_add_columns(
    balance_df, ["Operating Lease Liability"]
)
scorecard["Other Current Liabilites"] = combine_or_add_columns(
    balance_df, ["Other Current Liabilities"]
)
scorecard["Total Current Liabilities"] = balance_df["Total Current Liabilities"]
scorecard["Long Term Debt"] = combine_or_add_columns(
    balance_df, ["Long Term Debt", "Long Term Lease Liability"], replace_zero=True
)
scorecard["Other Long Term Liabilities"] = combine_or_add_columns(
    balance_df, ["Other Long Term Liabilities"]
)
scorecard["Total Liabilities"] = combine_or_add_columns(
    balance_df, ["Total Liabilities"]
)
scorecard["Additional Paid In Capital"] = combine_or_add_columns(
    balance_df, ["Additional Paid in Capital"]
)
scorecard["Retained Earnings"] = combine_or_add_columns(
    balance_df, ["Retained Earnings"]
)
scorecard["Total Stockholders Equity"] = combine_or_add_columns(
    balance_df, ["Total Stockholders Equity", "Total Equity(calc)"]
)
scorecard["Total Liabilities and Stockholders Equity"] = combine_or_add_columns(
    balance_df,
    ["Total Liabilities and Stockholders Equity", "Toal Liabilites and Equity(calc)"],
)
scorecard["Shares Outstanding (Diluted)"] = combine_or_add_columns(
    income_df,
    ["Weighted Average Shares Diluted", "Weighted Average Shares Outstanding"],
    replace_zero=True,
)
scorecard["Book Value Per Share"] = (
    scorecard["Total Stockholders Equity"] / scorecard["Shares Outstanding (Diluted)"]
)
scorecard["Operating Cash Flow"] = combine_or_add_columns(
    cash_df, ["Net Cash Provided by Operating Activities"]
)
scorecard["OCF/NI"] = scorecard["Operating Cash Flow"] / scorecard["Net Income"]
scorecard["Depreciation and Amortization"] = combine_or_add_columns(
    cash_df, ["Depreciation and Amortization", "Depreciation and Amortization(calc)"]
)
scorecard["Capex"] = negative(
    combine_or_add_columns(cash_df, ["Purchase of Property, Plant, and Equipment"])
)
scorecard["Capex / Depreciation"] = (
    scorecard["Capex"] / scorecard["Depreciation and Amortization"]
)
scorecard["Free Cash Flow"] = scorecard["Operating Cash Flow"] + scorecard["Capex"]
scorecard["Dividend Payment"] = negative(
    combine_or_add_columns(cash_df, ["Payment of Dividends"])
)
scorecard["Stock Repurchase"] = negative(
    combine_or_add_columns(cash_df, ["Repurchases of Common Stock"])
)
scorecard["ROA %"] = (scorecard["Net Income"] / scorecard["Total Assets"]) * 100
scorecard["ROE %"] = (
    scorecard["Net Income"] / scorecard["Total Stockholders Equity"]
) * 100
scorecard["Profit Margin %"] = scorecard["Net Income"] / scorecard["Net Revenue"] * 100
scorecard["Asset Turnover"] = scorecard["Net Revenue"] / scorecard["Total Assets"]
scorecard["Current Ratio"] = (
    scorecard["Total Current Assets"] / scorecard["Total Current Liabilities"]
)
scorecard["Equity Multiplier"] = (
    scorecard["Total Assets"] / scorecard["Total Stockholders Equity"]
)
scorecard["Net Working Capital"] = (
    scorecard["Total Current Assets"] - scorecard["Total Current Liabilities"]
)
scorecard["Debt to Equity Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Stockholders Equity"]
)
scorecard["Debt to Assets Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Assets"]
)
scorecard["Days Sales Outstanding"] = (
    scorecard["Accounts Receivable"] / scorecard["Net Revenue"]
) * 365
scorecard["Days of Inventory on Hand"] = (
    scorecard["Inventory"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Payables Period"] = (
    scorecard["Accounts Payable"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Receivables Turnover"] = (
    scorecard["Net Revenue"] / scorecard["Accounts Receivable"]
)
scorecard["Cash Conversion Cycle"] = (
    scorecard["Days Sales Outstanding"]
    + scorecard["Days of Inventory on Hand"]
    - scorecard["Payables Period"]
)

In [70]:
scorecard

,Net Revenue,Cost of Revenue,Gross Profit,Gross Margin,SG&A,Operating Income,Operating Margin %,Interest Expense,EBT,Tax Expense,...,Current Ratio,Equity Multiplier,Net Working Capital,Debt to Equity Ratio,Debt to Assets Ratio,Days Sales Outstanding,Days of Inventory on Hand,Payables Period,Receivables Turnover,Cash Conversion Cycle
2022,"26,974,000,000","11,618,000,000","15,356,000,000",0.57,"2,440,000,000","4,224,000,000",15.66,"262,000,000","4,181,000,000","-187,000,000",...,3.52,1.86,"16,510,000,000",0.86,0.46,51.79,162.08,37.48,7.05,176.38
2021,"26,914,000,000","9,439,000,000","17,475,000,000",0.65,"2,166,000,000","10,041,000,000",37.31,"236,000,000","9,941,000,000","189,000,000",...,6.65,1.66,"24,494,000,000",0.66,0.40,63.06,100.73,68.95,5.79,94.85
2020,"16,675,000,000","6,279,000,000","10,396,000,000",0.62,"1,940,000,000","4,532,000,000",27.18,"184,000,000","4,409,000,000","77,000,000",...,4.09,1.70,"12,130,000,000",0.70,0.41,53.17,106.15,66.79,6.86,92.52
2019,"10,918,000,000","4,150,000,000","6,768,000,000",0.62,"1,093,000,000","2,846,000,000",26.07,"52,000,000","2,970,000,000","174,000,000",...,7.67,1.42,"11,906,000,000",0.42,0.30,55.40,86.10,60.42,6.59,81.08
2018,"11,716,000,000","4,545,000,000","7,171,000,000",0.61,"991,000,000","3,804,000,000",32.47,"58,000,000","3,896,000,000","-245,000,000",...,7.94,1.42,"9,228,000,000",0.42,0.30,44.36,126.49,41.04,8.23,129.81
2017,"9,714,000,000","3,892,000,000","5,822,000,000",0.60,"815,000,000","3,210,000,000",33.05,"61,000,000","3,196,000,000","149,000,000",...,8.03,1.50,"8,102,000,000",0.50,0.34,47.53,74.65,55.89,7.68,66.29
2016,"6,910,000,000","2,847,000,000","4,063,000,000",0.59,"663,000,000","1,934,000,000",27.99,"58,000,000","1,905,000,000","239,000,000",...,4.77,1.71,"6,748,000,000",0.70,0.41,43.63,101.79,62.18,8.37,83.25
2015,"5,010,000,000","2,199,000,000","2,811,000,000",0.56,"602,000,000","747,000,000",14.91,"47,000,000","743,000,000","129,000,000",...,2.57,1.65,"3,702,000,000",0.63,0.38,36.79,69.38,49.13,9.92,57.04
2014,"4,682,000,000","2,083,000,000","2,599,000,000",0.56,"480,000,000","759,000,000",16.21,"46,000,000","755,000,000","124,000,000",...,6.38,1.63,"4,817,000,000",0.63,0.39,36.95,84.64,51.34,9.88,70.25
2013,"4,130,000,000","1,862,000,000","2,268,000,000",0.55,"436,000,000","496,000,000",12.01,"10,000,000","510,000,000","70,000,000",...,5.95,1.63,"4,679,215,000",NaN,NaN,37.68,76.01,63.59,9.69,50.10
